# **Projeto: Previsão de Intenção de Compra de Clientes em Loja Web**

**Descrição do Projeto**

Neste projeto, nosso objetivo é criar um sistema inteligente para antecipar a intenção de compra dos clientes em um site de e-commerce. Imagine poder prever quais clientes têm maior probabilidade de realizar compras online, com base em suas características e comportamentos passados. Essa capacidade de prever a intenção de compra não só aprimorará a experiência do cliente, mas também permitirá que a empresa direcione seus esforços de marketing de forma mais eficaz.

**Objetivo**

Queremos desenvolver um modelo preditivo capaz de analisar os padrões de comportamento dos clientes e identificar sinais que indicam a propensão deles para realizar compras no site da empresa. Para isso, vamos usar uma base de dados que contém informações detalhadas sobre os clientes, incluindo:

Dados demográficos (idade, renda, etc.)

Informações sobre compras anteriores



# Base de dados:

Year_Birth: Ano de nascimento do cliente.

Education: Nível de escolaridade do cliente.

Marital_Status: Estado civil do cliente.

Income: Renda anual da família do cliente.

Kidhome: Número de crianças na casa do cliente.

Recency: Número de dias desde a última compra do cliente.

Complain: 1 se o cliente reclamou nos últimos 2 anos, 0 caso contrário.

MntWines: Valor gasto em vinhos nos últimos 2 anos.

MntFruits: Valor gasto em frutas nos últimos 2 anos.

MntMeatProducts: Valor gasto em carnes nos últimos 2 anos.

MntFishProducts: Valor gasto em peixes nos últimos 2 anos.

MntSweetProducts: Valor gasto em doces nos últimos 2 anos.

MntGoldProds: Valor gasto em produtos de ouro nos últimos 2 anos.

NumDealsPurchases: Número de compras feitas com desconto

NumStorePurchases: Número de compras feitas diretamente nas lojas.

NumWebVisitsMonth: Número de visitas ao site da empresa no último mês.






**WebPurchases: Número de compras feitas pelo site da empresa.**

In [379]:
#Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform, loguniform, randint
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

# ETAPA 1:

**Preparação dos Dados**

**Exploração e Limpeza:** Analisar e limpar os dados para garantir que estejam prontos para a modelagem.

**Análise:** Construa uma storytelling com gráficos, analisando e retirando insights das informações.

In [345]:
#Carregando a base de dados
df = pd.read_csv('C:/Users/yurid/Downloads/marketing_campaign.csv', delimiter=';')
df


,Year_Birth,Education,Marital_Status,Income,Kidhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumStorePurchases,NumWebVisitsMonth,Complain,WebPurchases
0,1957,Graduation,Single,58138.0,0,58,635,88,546,172,88,88,4,7,0,1
1,1954,Graduation,Single,46344.0,1,38,11,1,6,2,1,6,2,5,0,0
2,1965,Graduation,Together,71613.0,0,26,426,49,127,111,21,42,10,4,0,1
3,1984,Graduation,Together,26646.0,1,26,11,4,20,10,3,5,4,6,0,0
4,1981,PhD,Married,58293.0,1,94,173,43,118,46,27,15,6,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,1967,Graduation,Married,61223.0,0,46,709,43,182,42,118,247,4,5,0,1
2236,1946,PhD,Together,64014.0,2,56,406,0,30,0,0,8,5,7,0,1
2237,1981,Graduation,Divorced,56981.0,0,91,908,48,217,32,12,24,13,6,0,0
2238,1956,Master,Together,69245.0,0,8,428,30,214,80,30,61,10,3,0,1


In [346]:
#Verificando valores nulos e os tipos de dados
print(df.isnull().sum())
print(df.info())

Year_Birth            0
Education             0
Marital_Status        0
Income               24
Kidhome               0
Recency               0
MntWines              0
MntFruits             0
MntMeatProducts       0
MntFishProducts       0
MntSweetProducts      0
MntGoldProds          0
NumStorePurchases     0
NumWebVisitsMonth     0
Complain              0
WebPurchases          0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year_Birth         2240 non-null   int64  
 1   Education          2240 non-null   object 
 2   Marital_Status     2240 non-null   object 
 3   Income             2216 non-null   float64
 4   Kidhome            2240 non-null   int64  
 5   Recency            2240 non-null   int64  
 6   MntWines           2240 non-null   int64  
 7   MntFruits          2240 non-null   int64  
 8   MntMeatProd

Conforme a análise acima, a variável renda apresenta valores faltantes. Como é a única variável que apresenta tal condição, se a média e a mediana foram próximas, os valores nulos serão substituídos pela mediana. Caso contrário, as linhas serão deletadas.

In [347]:
dados = {'media' : df['Income'].mean(),
         'mediana' : df['Income'].median(),
         'desvio' : df['Income'].std()}

for coluna, valor in dados.items():
    print(f'Valor da {coluna}: {valor}')

Valor da media: 52247.25135379061
Valor da mediana: 51381.5
Valor da desvio: 25173.076660901403


Dadas as discrepâncias entre média e mediana e desvio padrão alto, não faz sentido substituir os dados faltantes. Assim, eles serão deletados.

In [348]:
df.dropna(subset=['Income'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2216 entries, 0 to 2239
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year_Birth         2216 non-null   int64  
 1   Education          2216 non-null   object 
 2   Marital_Status     2216 non-null   object 
 3   Income             2216 non-null   float64
 4   Kidhome            2216 non-null   int64  
 5   Recency            2216 non-null   int64  
 6   MntWines           2216 non-null   int64  
 7   MntFruits          2216 non-null   int64  
 8   MntMeatProducts    2216 non-null   int64  
 9   MntFishProducts    2216 non-null   int64  
 10  MntSweetProducts   2216 non-null   int64  
 11  MntGoldProds       2216 non-null   int64  
 12  NumStorePurchases  2216 non-null   int64  
 13  NumWebVisitsMonth  2216 non-null   int64  
 14  Complain           2216 non-null   int64  
 15  WebPurchases       2216 non-null   int64  
dtypes: float64(1), int64(13), obj

In [349]:
#Verificando inconsistências nas variáveis categóricas
for variavel in ['Education', 'Marital_Status']:
    print(f'Valores únicos da coluna {variavel}: {df[variavel].unique()}')

Valores únicos da coluna Education: ['Graduation' 'PhD' 'Master' 'Basic' '2n Cycle']
Valores únicos da coluna Marital_Status: ['Single' 'Together' 'Married' 'Divorced' 'Widow' 'Alone' 'Absurd' 'YOLO']


Agora, vamos verificar as quantidades de cada valor de Marital_Status, a fim de verificar ruídos.

In [350]:
df['Marital_Status'].value_counts()

Marital_Status
Married     857
Together    573
Single      471
Divorced    232
Widow        76
Alone         3
Absurd        2
YOLO          2
Name: count, dtype: int64

Conforme os dados acima, 'Alone' e 'YOLO' podem significar o mesmo que 'single', enquanto 'Absurd' é inconclusivo. Dessa forma, as duas linhas com 'Absurd' serão deletadas, enquanto as demais opções citadas serão substituídas por 'Single'.

In [351]:
#Substituindo Alone e YOLO
df.replace({'Marital_Status':['Alone', 'YOLO']}, 'Single', inplace=True)

#Eliminando Absurd
df = df[df['Marital_Status']!='Absurd']

#Verificando novamente
df['Marital_Status'].value_counts()

Marital_Status
Married     857
Together    573
Single      476
Divorced    232
Widow        76
Name: count, dtype: int64

Após o tratamento da base, vamos realizar a verificação da distribuição das variáveis com histogramas e boxplots. De início, faremos de 4 em 4 com os histogramas em cima e os boxplots embaixo.

In [352]:
#Definindo as colunas em blocos e as cores:
colunas = df.columns
bloco = 4
b1 = colunas[:4]
b2 = colunas[4:8]
b3 = colunas[8:12]
b4 = colunas[12:16]
colors_1 = ['orchid', 'skyblue', 'yellow','tomato']
colors_2 = ['red', 'magenta', 'lime', 'olive']
colors_3 = ['brown', 'indigo','lawngreen', 'hotpink']
colors_4 = ['aqua', 'chartreuse', 'crimson', 'forestgreen']

#Plotando a primeira leva:
fig = make_subplots(cols=4, rows=2)

for i, coluna in enumerate(b1):
    fig.add_traces(go.Histogram(x=df[coluna],
                                marker_color=colors_1[i],
                                showlegend=False),
                   rows = 1, cols = i+1)
    fig.add_traces(go.Box(y=df[coluna],
                          marker_color=colors_1[i],
                          name= coluna),
                   rows = 2, cols = i+1)

fig.update_layout(bargap = 0.2,
                  title_text = 'Variáveis do grupo 1')

Conforme os gráficos acima, alguns insigths podem ser retirados. De início, a variável Year_Birth apresenta dois outilers importantes: 1893 e 1900, ja que as chances de haverem pessoas vivas que nasceram nesses anos é mínima. Assim, convém eliminar as duas linhas que contém esses dados. No caso das variáveis educação e estado civil, nenhum outlier foi detectado, ainda que não haja uniformidade de distribuição entre as classes. Por fim, a variável renda será filtrada pelo método do intervalo interquartil, de forma a eliminar os outliers mostrados no boxplot.

In [353]:
#Filtrando os outliers de Year_Birth
df = df[df['Year_Birth']>1900]

#Filtrando os outliers de renda (IQR)
q1 = df['Income'].quantile(0.25)
q3 = df['Income'].quantile(0.75)
IQR = q3-q1

lim_min = q1 - 1.5*IQR
lim_max = q3 + 1.5*IQR

df = df[df['Income']>lim_min]
df = df[df['Income']<lim_max]

#Verificando as novas distribuições das variáveis alteradas
fig = make_subplots(rows=2, cols=2)

for i, coluna in enumerate(['Year_Birth', 'Income']):
    fig.add_traces(go.Histogram(x=df[coluna],
                                showlegend = False,
                                marker_color=colors_1[i]),
                   rows = 1, cols = i+1)
    fig.add_traces(go.Box(y=df[coluna],
                          marker_color=colors_1[i],
                          name = coluna),
                   rows = 2, cols = i+1)

fig.update_layout(bargap = 0.2,
                  title_text = 'Distribuição das variáveis após alteração')
fig.show()

Após o filtro de outliers, temos que a distribuição das variáveis está próxima à normal. Agora, seguiremos para o próximo bloco.

In [354]:
fig = make_subplots(cols=4, rows=2)

for i, coluna in enumerate(b2):
    fig.add_traces(go.Histogram(x=df[coluna],
                                marker_color=colors_2[i],
                                showlegend=False),
                   rows = 1, cols = i+1)
    fig.add_traces(go.Box(y=df[coluna],
                          marker_color=colors_2[i],
                          name= coluna),
                   rows = 2, cols = i+1)

fig.update_layout(bargap = 0.2,
                  title_text = 'Variáveis do grupo 2')

In [355]:
fig = make_subplots(cols=4, rows=2)

for i, coluna in enumerate(b3):
    fig.add_traces(go.Histogram(x=df[coluna],
                                marker_color=colors_3[i],
                                showlegend=False),
                   rows = 1, cols = i+1)
    fig.add_traces(go.Box(y=df[coluna],
                          marker_color=colors_3[i],
                          name= coluna),
                   rows = 2, cols = i+1)

fig.update_layout(bargap = 0.2,
                  title_text = 'Variáveis do grupo 3')

No caso do grupo 2, tem-se que, para o número de filhos, a maioria tem nenhum ou apenas um filho. No que tange ao número de dias desde a última compra, tem-se um resultado bem uniforme. Por fim, para o total gasto em vinhos e frutas (Grupo 2) e carne, peixes, doces e ouro (Grupo 3) tem-se um comportamento esperado de uma população: há mais indivíduos que gastam menos do que indivíduos que gastam mais. Diante disso, não haverá alterações para estas variáveis.

In [356]:
fig = make_subplots(cols=4, rows=2)

for i, coluna in enumerate(b4):
    fig.add_traces(go.Histogram(x=df[coluna],
                                marker_color=colors_4[i],
                                showlegend=False),
                   rows = 1, cols = i+1)
    fig.add_traces(go.Box(y=df[coluna],
                          marker_color=colors_4[i],
                          name= coluna),
                   rows = 2, cols = i+1)

fig.update_layout(bargap = 0.2,
                  title_text = 'Variáveis do grupo 4')

No grupo 4, o número de compras na loja está bem distribuído, enquanto a variável target encontra-se balanceada. No que tange ao número de visitas mensais, temos alguns outliers, que serão retirados pelo método do intervalo interquartil. Por fim, a variável 'Complain' apresenta uma resposta diferente apenas, de forma que não contribuirá significativamente para o modelo. Assim, será retirada por completo.

In [357]:
#Retirando a coluna 'Complain'
df.drop(columns = 'Complain',  inplace = True)

#Filtrando os outliers de NumWebVisitsMonth
q1_web = df['NumWebVisitsMonth'].quantile(0.25)
q3_web = df['NumWebVisitsMonth'].quantile(0.75)
IQR_web = q3_web - q1_web

lim_min_web = q1_web - 1.5*IQR_web
lim_max_web = q3_web + 1.5*IQR_web
df = df[df['NumWebVisitsMonth']>lim_min_web]
df = df[df['NumWebVisitsMonth']<lim_max_web]

#Verificando a nova distribuição da variável
fig = make_subplots(rows = 1, cols=2)

fig.add_traces(go.Histogram(x=df['NumWebVisitsMonth'],
                            marker_color=colors_4[1],
                            showlegend=False),
               rows = 1, cols = 1)
fig.add_traces(go.Box(y=df['NumWebVisitsMonth'],
                      marker_color=colors_4[1],
                      name = 'NumWebVisitsMonth'),
               rows = 1, cols = 2)
fig.update_layout(bargap = 0.2, title_text = 'Nova distribuição da variável NumWebVisitsMonth')
fig.show()

Dada a nova distribuição, agora vamos investigar as relações entre as variáveis. Diante disso, busca-se responder às seguintes perguntas:
- Há relação entre renda e número de visitas mensais?
- Há relação entre renda e número de compras?
- Há relação entre ano de nascimento e renda?

In [358]:
#Verificando as relações a partir das médias agrupadas
mean_income_webvis = df['Income'].groupby(df['NumWebVisitsMonth']).mean().reset_index()
mean_income_numcom = df['Income'].groupby(df['NumStorePurchases']).mean().reset_index()
mean_income_year = df['Income'].groupby(df['Year_Birth']).mean().reset_index()

#Definindo as cores dos gráficos
colors_bivar = ['indigo', 'royalblue', 'lawngreen']

#Plotando os gráficos
dfs = [mean_income_webvis, mean_income_numcom, mean_income_year]
cols_x = [dfa.columns[0] for dfa in dfs]

fig = make_subplots(rows = 1,
                   cols = 3,
                   subplot_titles = ['Renda x Número de Visitas',
                                     'Renda x Compras',
                                     'Renda x Ano'])

for i, (var, col) in enumerate(zip(dfs, cols_x)):
    fig.add_traces(go.Bar(x=var[col],
                          y= var['Income'],
                          marker_color=colors_bivar[i],
                          showlegend=False),
                   rows = 1, cols = i+1)


fig.update_layout(bargap=0.2)

fig.show()


Dos gráficos analisados, temos que:
- Quanto maior a renda, menor o número de visitas;
- Quanto maior a renda, maior o número de compras;
- O ano de nascimento não tem tanta influência na renda.

# ETAPA 2:
**Pré-processamento**

**Análise Correlação:** Verifique a correlação entre as váriaveis e análise se há espaço para retirar váriaveis que não te parecem importantes.

**Codificação de Variáveis Categóricas:** Transformar variáveis categóricas em um formato que os modelos de machine learning possam interpretar.


**Separe a base em Y, X e Treino e teste:**: Faça a separação da base.

**Realize a padronização dos dados**: Padronize os dados para garantir eficiência no modelo e eficácia.



Para a variável Marital Status será utilizado o one-hot encoding, que garante que não haja pressuposição de ordem entre as variáveis. No caso da variável Education, será utilizado ordinal encoding, já que há uma ordem pré-estabelecida.

In [359]:
#Variável Marital Status (One-Hot)
df = pd.get_dummies(df, columns = ['Marital_Status'], dtype = int)


#Variável Education (Ordinal)
mapping = {'Basic' : 0,
           '2n Cycle': 1,
           'Graduation': 2,
           'Master':3,
           'PhD':4}

df['Education'] = df['Education'].map(mapping)

#Verificando o novo df
df

,Year_Birth,Education,Income,Kidhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumStorePurchases,NumWebVisitsMonth,WebPurchases,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow
0,1957,2,58138.0,0,58,635,88,546,172,88,88,4,7,1,0,0,1,0,0
1,1954,2,46344.0,1,38,11,1,6,2,1,6,2,5,0,0,0,1,0,0
2,1965,2,71613.0,0,26,426,49,127,111,21,42,10,4,1,0,0,0,1,0
3,1984,2,26646.0,1,26,11,4,20,10,3,5,4,6,0,0,0,0,1,0
4,1981,4,58293.0,1,94,173,43,118,46,27,15,6,5,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,1967,2,61223.0,0,46,709,43,182,42,118,247,4,5,1,0,1,0,0,0
2236,1946,4,64014.0,2,56,406,0,30,0,0,8,5,7,1,0,0,0,1,0
2237,1981,2,56981.0,0,91,908,48,217,32,12,24,13,6,0,1,0,0,0,0
2238,1956,3,69245.0,0,8,428,30,214,80,30,61,10,3,1,0,0,0,1,0


In [360]:

#Plotando a matriz de correlação
cor = df.corr()

fig = go.Figure()
fig.add_traces(go.Heatmap(x=cor.columns,
                          y=cor.index,
                          z=cor,
                          colorscale = 'purd',
                          texttemplate='%{z}'))
fig.update_layout(title_text = 'Matriz de Correlação')
fig.show()

Da matriz de correlação, temos que as variáveis relacionadas ao estado civil dos indivíduos apresentam correlação forte somente entre si, de forma que é possível retirá-las das próximas análises. Além disso, as variáveis Recency, Year_Birth e Education apresentam somente valores baixos, então também serão retiradas.

In [361]:
#Retirando as variáveis
df.drop(columns =['Year_Birth', 'Education', 'Recency', 'Marital_Status_Divorced', 'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Together','Marital_Status_Widow'], inplace = True)

#Verificando o novo df
df

,Income,Kidhome,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumStorePurchases,NumWebVisitsMonth,WebPurchases
0,58138.0,0,635,88,546,172,88,88,4,7,1
1,46344.0,1,11,1,6,2,1,6,2,5,0
2,71613.0,0,426,49,127,111,21,42,10,4,1
3,26646.0,1,11,4,20,10,3,5,4,6,0
4,58293.0,1,173,43,118,46,27,15,6,5,1
...,...,...,...,...,...,...,...,...,...,...,...
2235,61223.0,0,709,43,182,42,118,247,4,5,1
2236,64014.0,2,406,0,30,0,0,8,5,7,1
2237,56981.0,0,908,48,217,32,12,24,13,6,0
2238,69245.0,0,428,30,214,80,30,61,10,3,1


In [362]:
#Plotando novamente a matriz de correlação
cor_2 = df.corr()

fig = go.Figure()

fig.add_traces(go.Heatmap(x=cor_2.columns,
                          y=cor_2.index,
                          z=cor_2,
                          colorscale = 'purd',
                          texttemplate='%{z}'))

fig.update_layout(title_text = 'Matriz de Correlação Aprimorada')

fig.show()

In [383]:
#Separando em x e y
y=df['WebPurchases']
x = df.drop(columns = ['WebPurchases'])

#Padronizando a base X
standard = StandardScaler()
x= standard.fit_transform(x)


In [386]:
# PCA para redução da dimensionalidade
pca=PCA(n_components=8)
pca.fit(x)

#Definindo as variáveis para o gráfico
expl_var = pca.explained_variance_ratio_
cumul_val = expl_var.cumsum()

#Plotando o gráfico da PCA
fig = go.Figure()

fig.add_traces(go.Scatter(x=[1,2,3,4,5,6,7,8],
                          y=cumul_val,
                          mode='lines+markers',
                          line=dict(dash='dash',
                                    color= 'crimson'),
                          marker=dict(color = 'tomato',
                                      size = 10)))

fig.update_layout(title_text='Variância acumulada por componente',
                  xaxis_title='Componentes',
                  yaxis_title = 'Variância Explicada')

fig.show()


Conforme o gráfico, levando em conta a redução de dimensionalidade, temos que a PCA só foi eficaz a partir de 8 componentes. Como temos 10 variáveis, seguiremos com os dados normais. Em seguida, serão testados os modelos Regressão Logística e Random Forest.

In [387]:
#Separando as bases em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

# ETAPA 3:

**Modelagem**

Escolha ao menos 2 técnicas de machine learning e rode 2 modelos, afim de identificar qual tem o melhor resultado para essa base. Lembrando que estamos lidando com uma classificação binária.

In [388]:
#Aplicando a Regressão Logística
reglog = LogisticRegression()

#Utilizando Random Search para melhoria dos parâmetros do modelo.
param_distributions = {
    'C': loguniform(1e-4, 1e3),
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver' : ['saga'],
    'l1_ratio' : np.linspace(0.1, 1.0, 10)
}

# Configurando o Random Search com cross validation
random_search_RL = RandomizedSearchCV(
    reglog,
    param_distributions=param_distributions,
    n_iter=100,
    cv=5,
    n_jobs=-1,
    random_state=42,
    error_score='raise'
)

#Rodando o modelo com os dados de treino
random_search_RL.fit(x_train, y_train)

,estimator,LogisticRegression()
,param_distributions,"{'C': <scipy.stats....0020822EEF200>, 'l1_ratio': array([0.1, 0....8, 0.9, 1. ]), 'penalty': ['l1', 'l2', ...], 'solver': ['saga']}"
,n_iter,100
,scoring,None
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,'raise'


In [390]:
#Aplicando a Random Forest
rf = RandomForestClassifier()

#Utilizando Random Search
param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': [None, 10, 20, 30, 50, 70, 100],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

#Configurando Random Search com cross validation
random_search_RF = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=100,
    cv=5,
    n_jobs=-1,
    random_state=42,
    error_score='raise'
)

#Rodando o modelo
random_search_RF.fit(x_train, y_train)

,estimator,RandomForestClassifier()
,param_distributions,"{'bootstrap': [True, False], 'criterion': ['gini', 'entropy'], 'max_depth': [None, 10, ...], 'max_features': ['sqrt', 'log2', ...], ...}"
,n_iter,100
,scoring,None
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,'raise'


In [392]:
#Extraindo os melhores parâmetros e melhor pontuação
best_params = {'RegLog': [random_search_RL.best_params_, random_search_RL.best_score_],
               'RandomForest': [random_search_RF.best_params_, random_search_RF.best_score_]}

print(best_params)

{'RegLog': [{'C': np.float64(763.1610566529841), 'l1_ratio': np.float64(0.9), 'penalty': 'elasticnet', 'solver': 'saga'}, np.float64(0.8495114006514658)], 'RandomForest': [{'bootstrap': True, 'criterion': 'gini', 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 195}, np.float64(0.9009771986970684)]}


# ETAPA 4:

**Avaliação**

Avalie os resultados encontrados nos dois modelos e identifique qual te pareceu realizar melhor as previsões.

Utilize além das métricas padrões a matriz de confusão.

In [404]:
#Previsões Regressão Logística
y_pred_rl = random_search_RL.predict(x_test)

#Previsões Random Forest
y_pred_rf = random_search_RF.predict(x_test)

#Relatório Regressão Logística
relatorio_rl = classification_report(y_test, y_pred_rl)

#Relatório Random Forest
relatorio_rf = classification_report(y_test, y_pred_rf)

#Matriz de confusão Regressão Logística
conf_mat_rl = confusion_matrix(y_test, y_pred_rl)

#Matriz de confusão Random Forest
conf_mat_rf = confusion_matrix(y_test, y_pred_rf)

#Regressão Logística: Análise dos resultados
fig = go.Figure()

fig.add_traces(go.Heatmap(x=[0,1],
                          y = [0,1],
                          z= conf_mat_rl,
                          colorscale = 'teal',
                          texttemplate = '%{z}',
                          showscale=False))

fig.update_layout(title_text = 'Matriz de Confusão da Regressão Logística')
fig.show()

print('Relatório da Regressão Logística:')
print(relatorio_rl)

Relatório da Regressão Logística:
              precision    recall  f1-score   support

           0       0.89      0.86      0.88       333
           1       0.87      0.90      0.88       326

    accuracy                           0.88       659
   macro avg       0.88      0.88      0.88       659
weighted avg       0.88      0.88      0.88       659



Para a regressão logística, tanto a acurácia geral do modelo, quanto a precisão e o recall estão em 88%, o que denota um bom desempenho. Todavia, o recall da classe 1 é mais alto que o da classe 0, o que significa que o modelo tem maior facilidade para 'não deixar passar' clientes compradores. Além disso, o f1-score denota que o modelo mantém a consistência nas previsões. No caso da matriz de confusão, temos apenas 34 erros para a classe zero contro 45 da classe um, o que é evidenciado pela precisão maior para a classe zero. Temos, então, um desempenho excelente, que agora será comparado à Random Forest.

In [406]:
#Random Forest: Análise dos resultados
fig = go.Figure()

fig.add_traces(go.Heatmap(x=[0,1],
                          y = [0,1],
                          z= conf_mat_rf,
                          colorscale = 'greens',
                          texttemplate = '%{z}',
                          showscale=False))

fig.update_layout(title_text = 'Matriz de Confusão da Random Forest')
fig.show()

print('Relatório da Random Forest:')
print(relatorio_rf)

Relatório da Random Forest:
              precision    recall  f1-score   support

           0       0.98      0.89      0.93       333
           1       0.90      0.98      0.94       326

    accuracy                           0.93       659
   macro avg       0.94      0.93      0.93       659
weighted avg       0.94      0.93      0.93       659



Para a Random Forest, o modelo provou-se ainda mais sensível, com precisão de 94% e recall, acurácia e f1-score de 93%. Mais uma vez, tem-se maior recall para a classe 1 e mais precisão na classe 0, o que é reforçado pela matriz de confusão. Diante do analisado, o desempenho da Random Forest foi superior ao da regressão logística. Todavia, é um modelo com maior custo computacional. Em caso de priorização da velocidade das análises, a regressão logística é uma saída ideal. Porém, caso a prioridade seja precisão e acurácia, sugire-se a implementação do Random Forest. 